In [1]:
from util import load_data, fdr, plot_report
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from collections import ChainMap
import matplotlib.pyplot as plt

In [2]:
x_train, x_test, x_oot, y_train, y_test, y_oot = load_data('var_30.csv')
x_train.head()

,ssn_name_count_14,ssn_name_dob_count_14,ssn_lastname_count_14,ssn_firstname_count_14,ssn_dob_count_0_by_30,name_dob_count_0_by_30,fulladdress_homephone_count_14,fulladdress_count_1_by_14,fulladdress_homephone_day_since,ssn_dob_day_since,...,fulladdress_count_0,name_dob_count_3,fulladdress_homephone_count_0_by_14,name_day_since,name_count_14,ssn_dob_count_0_by_14,ssn_lastname_count_0_by_14,ssn_firstname_count_0_by_14,ssn_name_day_since,address_count_1_by_7
68402,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-1.339046,-1.345330,...,-0.041351,-0.046415,0.120387,-1.120304,-0.159376,0.118004,0.123458,0.123528,-1.331010,0.103441
184696,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.921328,-0.927872,...,-0.041351,-0.046415,0.120387,-0.997412,-0.159376,0.118004,0.123458,0.123528,-0.912606,0.103441
232527,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.759937,-0.766581,...,-0.041351,-0.046415,0.120387,-0.827253,-0.159376,0.118004,0.123458,0.123528,-0.750949,0.103441
308441,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.494116,-0.500926,...,-0.041351,-0.046415,0.120387,-0.278963,-0.159376,0.118004,0.123458,0.123528,-0.484692,0.103441
183403,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.930821,-0.937360,...,-0.041351,-0.046415,0.120387,-0.713814,-0.159376,0.118004,0.123458,0.123528,-0.922115,0.103441


In [3]:
dt = DecisionTreeClassifier()
params = {'min_samples_leaf': [64, 128, 256, 512],
          'max_depth' : [10, 20, 40, 80],
          'ccp_alpha': [0, 1e-8, 1e-7, 5e-7, 1e-6]}
grid = GridSearchCV(dt, params, cv=10, refit='FDR', 
                    verbose=1, n_jobs=8,
                    scoring={'FDR': fdr})
grid.fit(x_train, y_train)

Fitting 10 folds for each of 80 candidates, totalling 800 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   20.5s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  2.6min
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:  6.3min
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed: 12.1min
[Parallel(n_jobs=8)]: Done 800 out of 800 | elapsed: 12.3min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=8,
             param_grid={'ccp_alpha': [0, 1e-08, 1e-07, 5e-07, 1e-06],
         

In [4]:
print(f"The best parameters are {grid.best_params_} "
      f"with a score of {grid.best_score_:0.5f}")

The best parameters are {'ccp_alpha': 0, 'max_depth': 20, 'min_samples_leaf': 128} with a score of 0.52200


In [5]:
pd.set_option('display.max_rows', 999)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_FDR').filter(
        items=['param_max_depth', 'param_min_samples_leaf',
               'param_ccp_alpha', 'mean_test_FDR', 'std_test_FDR',
               'rank_test_FDR', 'mean_test_ROC', 'std_test_ROC'])

,param_max_depth,param_min_samples_leaf,param_ccp_alpha,mean_test_FDR,std_test_FDR,rank_test_FDR
5,20,128,0,0.521997,0.015814,1
37,20,128,1e-07,0.521996,0.013704,2
21,20,128,1e-08,0.521888,0.015748,3
38,20,256,1e-07,0.521780,0.014855,4
41,40,128,1e-07,0.521345,0.015244,5
45,80,128,1e-07,0.521345,0.015244,5
6,20,256,0,0.520476,0.013628,7
22,20,256,1e-08,0.520476,0.013628,7
30,80,256,1e-08,0.519933,0.014682,9
10,40,256,0,0.519933,0.014682,9


In [6]:
dt = DecisionTreeClassifier(ccp_alpha=1e-07, max_depth=40, min_samples_leaf=128)
params = {'class_weight': ['balanced', None],
          'criterion': ['gini', 'entropy']}
grid_2 = GridSearchCV(dt, params, cv=10, refit='FDR', 
                      verbose=1, n_jobs=8,
                      scoring={'FDR': fdr, 'ROC': 'roc_auc'})
grid_2.fit(x_train, y_train)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  40 out of  40 | elapsed:   46.7s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=1e-07,
                                              class_weight=None,
                                              criterion='gini', max_depth=40,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=128,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=8,
             param_grid={'class_

In [7]:
print(f"The best parameters are {grid_2.best_params_} "
      f"with a score of {grid_2.best_score_:0.5f}")

The best parameters are {'class_weight': None, 'criterion': 'gini'} with a score of 0.52134


In [8]:
pd.DataFrame(grid_2.cv_results_).filter(
    items=['param_class_weight', 'param_criterion',
           'mean_test_FDR', 'std_test_FDR', 'rank_test_FDR',
           'mean_test_ROC', 'std_test_ROC'])

,param_class_weight,param_criterion,mean_test_FDR,std_test_FDR,rank_test_FDR,mean_test_ROC,std_test_ROC
0,balanced,gini,0.519717,0.014567,3,0.766285,0.008917
1,balanced,entropy,0.516674,0.014434,4,0.765267,0.009924
2,None,gini,0.521345,0.015244,1,0.767063,0.007643
3,None,entropy,0.520042,0.014584,2,0.766137,0.009704


In [9]:
dt = DecisionTreeClassifier(ccp_alpha=1e-07, 
                            max_depth=40, 
                            min_samples_leaf=128, 
                            class_weight=None,
                            criterion='gini').fit(x_train, y_train)
print(fdr(dt, x_train, y_train))
print(fdr(dt, x_test, y_test))
print(fdr(dt, x_oot, y_oot))

0.5395894428152492
0.5222958057395144
0.5302978432043821


In [10]:
x_train_40, x_test_40, x_oot_40, y_train_40, y_test_40, y_oot_40 = load_data('var_40.csv')
dt = DecisionTreeClassifier(ccp_alpha=1e-07, 
                            max_depth=40, 
                            min_samples_leaf=128, 
                            class_weight=None,
                            criterion='gini').fit(x_train_40, y_train_40)
print(fdr(dt, x_train_40, y_train_40))
print(fdr(dt, x_test_40, y_test_40))
print(fdr(dt, x_oot_40, y_oot_40))

0.5737186477644493
0.5629887054735013
0.5556316330023965
